### The cell below contains a loop for finding the mean temp/rainfall in a user defined area around a specified long/lat

In [ ]:
# Importing needed modules for reading netCDF4 files etc...

from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import os

# As this data was manually collated due to the nature of the raw data the long/lat of the collection sites were
# also entered manually.

# Woolpit,Croft,Ellon,Wellbourne,East Malling, Dumlow
# 1       2       3     4           5           6

someplaceslat =[52.210085,54.481952,57.418575,53.077383,51.137781,51.898061 ] 
someplaceslong= [1.110683, -1.555016, -2.098217, -0.566611,0.444285, 0.374528]

# Creating lists of all rainfall and temperature files. (For this data set it will only use 2010 onwards)

tempfiles=glob.glob(os.getcwd()+'/Data/tas*.nc')
rainfiles=glob.glob(os.getcwd()+'/Data/rainfall*.nc')


# A simple counter that will be used for labelling npy files etc...

regioncounter = 0 



# This first loop iterates through the manually entered long and lat values.

for loc1,loc2 in zip(someplaceslong,someplaceslat):
    
    
    # This counter is used to check whether it is the first time a specific long and lat has been used.
    # This is so the process simply use the same points at a later date instead of filtering all points used.
    
    loopcounter = 0
    
    # Declaring lists that will be used to store the average temps, rain and also the positions of the measurements
    # within the user defined shape about the collection sites.
    
    alltempsforplotting,allrainsforplotting = [],[]
    
    posxs,posys = [],[]
    
    regioncounter +=1
    
    # Now the first long and lat has been specified the following loop will access all temp/rain files found earlier
    
    for file,file2 in zip(tempfiles,rainfiles):
        
        # The following block opens the netCDF4 rain/temp files and extracts the rain and temperature data
        # along with the long and lat.
        
        dataset = Dataset(file)
        Temps = dataset.variables['tas'][:]
        lat = dataset.variables['latitude'][:]
        long = dataset.variables['longitude'][:]
        datasetrain = Dataset(file2)
        Rains = datasetrain.variables['rainfall'][:]
        
        # The following nested loop statement goes through the netCDF4 files. Each file contains 12 months of data
        # for about 1 million lat/long combinations. It iterates through those and only collects points within
        # and area of long/lat the user has specified below. It then saves the position of these files in the array

        if loopcounter == 0:
            for counter in range(0,len(Temps)):
                for posx,(longlist,latlist) in enumerate(zip(long,lat)):
                    for posy,(item1,item2) in enumerate(zip(longlist,latlist)):
                        if item1 > (loc1 - 0.25) and item1 < (loc1 +0.25) and item2 > (loc2 - 0.25) and item2 < (loc2 +0.25):
                            posys.append(posy)
                            posxs.append(posx)
                            
                            
                # Now the code has found the relevant co-ordinates the temp/rain data from those co-ordinates are 
                # appended to a list for averaging. 

                z = []
                for stuff,stuff2 in zip(posxs,posys):
                    z.append(Temps[counter,stuff,stuff2])

                z2 = []
                for stuff,stuff2 in zip(posxs,posys):
                    z2.append(Rains[counter,stuff,stuff2])

                z = np.array(z)
                mu = np.nanmean(z)
                print(mu,'Temp')
                alltempsforplotting.append(mu)

                z2 = np.array(z2)
                mu2 = np.nanmean(z2)
                print(mu2, 'Rainfall')
                allrainsforplotting.append(mu2)
                
        # This was a quick optimization that checks whether the co-ordinates of the data have been found before
        # and if so skip the filtering process and just append those data points.

        elif loopcounter != 0 :


            for counter in range(0,len(Temps)):

                z = []
                for stuff,stuff2 in zip(posxs,posys):
                    z.append(Temps[counter,stuff,stuff2])

                z2 = []
                for stuff,stuff2 in zip(posxs,posys):
                    z2.append(Rains[counter,stuff,stuff2])

                z = np.array(z)
                mu = np.nanmean(z)
                print(mu,'Temp')
                alltempsforplotting.append(mu)

                z2 = np.array(z2)
                mu2 = np.nanmean(z2)
                print(mu2, 'Rainfall')
                allrainsforplotting.append(mu2)
        loopcounter += 1 
        
    print('1st region done')
    
    
    # Now the temp/rain data has been collected the appropriate yield data is opened from a csv file.
    
    x = pd.read_csv(os.getcwd()+'/Data/' + str(regioncounter)+'.csv')
    x= np.array(x)
    
    # The mean of the yield is found so the anomalies can be calculated. Two arrays are created for plotting against
    # months/years
    
    timeyield = np.linspace(0,7,8)
    time = np.linspace(0,96,96)

    MuYield = np.mean(x[:,1])
    
    
    # For both rain and temperature a rolling mean is created. This is done over 3 months as specified in the 
    # FACYNation paper. This is then used to create the anomalies and plotted.

    s = pd.Series(alltempsforplotting)

    tempsrolling = s.rolling(3,min_periods=1).mean()

    plt.scatter(time,alltempsforplotting-tempsrolling)


    s = pd.Series(allrainsforplotting)

    rainsrolling = s.rolling(3, min_periods=1).mean()

    plt.scatter(time,allrainsforplotting-rainsrolling)
    plt.show()


    
    plt.scatter(timeyield*12,x[:,1]-MuYield)
    
    
    
    # All the data is saved to region specific npy files for later use in the adapted FACYnation codes.

    np.save('TempAnom'+str(regioncounter),alltempsforplotting-tempsrolling)

    np.save('RainAnom'+str(regioncounter),allrainsforplotting-rainsrolling)

    np.save('YieldAnom'+str(regioncounter),x[:,1]-MuYield)


    np.save('Yield'+str(regioncounter),x[:,1])
    np.save('Rain'+str(regioncounter),allrainsforplotting)
    np.save('Temps'+str(regioncounter),alltempsforplotting)


### The original code for plotting and saving the data. This has been integrated into the code above but has been left here for debugging

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

x = pd.read_csv(os.getcwd()+'/Data/6.csv')
x= np.array(x)

timeyield = np.linspace(0,7,8)
time = np.linspace(0,96,96)

MuYield = np.mean(x[:,1])

s = pd.Series(alltempsforplotting)

tempsrolling = s.rolling(3,min_periods=1).mean()

plt.scatter(time,alltempsforplotting-tempsrolling)


s = pd.Series(allrainsforplotting)

rainsrolling = s.rolling(3, min_periods=1).mean()

plt.scatter(time,allrainsforplotting-rainsrolling)
plt.show()


plt.scatter(timeyield*12,x[:,1]-MuYield)



np.save('TempAnom',alltempsforplotting-tempsrolling)

np.save('RainAnom',allrainsforplotting-rainsrolling)

np.save('YieldAnom',x[:,1]-MuYield)


np.save('Yield',x[:,1])
np.save('Rain',allrainsforplotting)
np.save('Temps',alltempsforplotting)


